# Python Zen

In [1]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


In [28]:
import sys    # нужно для sys.getsizeof(объект) - чтобы сравнивать размеры в памяти

# Основные понятие
- Приоритеты операций
- Динамическая типизация
- Объекты
- None
- Time complexity
- Refactoring
- ОПП: наследование, инкапсуляция, полиморфизм

**Приоритеты операций**: ветвление (строчное) < логика: or < and > in/is/not < сравнения < =/!=  < битовые < математика < индексация < cрезы < функции < генераторы списков

**Динамическая типизация** - автоматическое связывание переменной (иначе называется ссылка) и объекта. Т.е. объявляя операцию присвоения (напр., a=3), python (концептуально) делает 3 действия: 
- Создается объект в памяти (напр., цел. число 3). Объект имеет тип данных.
- Создается переменная (перемен. a). Не имеет типа, по сути является ссылкой на объект.
- Переменной дается ссылка на объект определенного типа.

Нужно это для полиморфизма python. Это дает языку гибкость с написания сценария (часто устраняет необходимость писать код в расчете на определенный тип данных).   

---
Фичи:
- Так операция присвоения передает ссылку, поэтому вызов переменной без присвоенного значения = ошибка.
- Повторное присвоение переменно какого-либо объекта просто передаст новую ссылку -> не имеет значение какие типы данных были у первого и второго объектов (до и после).
- Каждый раз обращаясь к переменной/ссылке, python обращается к объекту переходя по его ссылке.
- Переменную также называют ссылкой. 
- Каждое новое действие (напр., мат. операция) создает в памяти новый объект (даже если не было операции присвоения -> нет ссылки на этот объект). Однако, если такой объект уже был создан (напр., цел.число 3), то python просто обратиться у нему (т.е. один объект может иметь много ссылок на себя). Если на него нет ссылок, то объект удаляется автоматически, чтобы не тратить память.


**Объекты** - элементы памяти, выделенные под определенный тип данных. Каждый объект хранит в себе: 1. описатель типа; 2. счетчик ссылок.  

Пример: Цел. числ. 3: создается объект который содержит 1. указатель в памяти на int 3 (значение); 2. указатель на тип int (описатель типа, те в памяти создается объект int и на него ссылаются все int в программе); 3. Счетчик ссылок=1  

---
Фичи:
- Когда счетчик ссылок равен 0, интерпретатор python включает garbage collection: удаляет объект и высвобождает память. Объект не будет удален, пока на него ссылается ссылается хотя бы одна переменная.
- При компиляции (.pyc) создаются популярные объекты, на которые часто ссылаются (напр., 0)
- Если сказано, что объект не именяем - это значит, что *его элемет не может быть переписан*. Это не значит, что сам объекты не может менять размер (напр., с помощью методов)


**None** - пустой тип данных. Вывоводится, когда у функции нет `return`. Или используется `assert` 

**Time complexity** [for python objects](https://wiki.python.org/moin/TimeComplexity)

**Code refactoring** - процесс разбиения существующего кода на более удобные фунциональные блоки (функции, классы и т.д.), не меняя при этом внутреннюю работу программы. Т.е. это оптимизация кода. 

Класс реализуются на основе уже существующих объектов (в тч классов) – принцип **Наследования**. То что все функции внутри класса, скрыты отглаз и реализованы как методы – **Инкапсуляция**. **Полиморфизм** (способность применять одинаковые методы на разных классах) – прямое следствие наследования в классах.   

# Объектная структура Питона

Как известно, все в питоне является объектом. 
- В интерпретаторе CPython все реализовано через структуры (`struct`).
- Любой объект (в т.ч. типы данных, ф-ции и т.д.) реализованны через наследования от более базового типа. 
- Под капотом любого объекта в питоне лежит одна из 2х базовых структур: `PyObject` (для неизменяемых объектов) и `PyVarObject` (для изменяемых объектов).

## PyObject
Стуктура для неизменяемых объектов.
```c
typedef struct _object {
    Py_ssize_t ob_refcnt;
    struct _typeobject *ob_type;
} PyObject;
```
У этой структуры 2 поля:
- `ob_refcnt` - счетчик ссылок. Он следит за кол-вом ссылок на данных объект. Когда он будет = 0, garbage collector удалит этот объект.
- `*ob_type` - указатель на тип объекта.

## PyVarObject
Стуктура для изменяемых объектов.
```c
typedef struct {
    PyObject ob_base;
    Py_ssize_t ob_size; // Кол-во элементов в переменной части
} PyVarObject;
```
- `ob_base` - объект типа PyObject (т.к. у каждого объекта в питоне есть счетчик ссылок и указатель на тип объекта.)
- `ob_size` - счетчик элементов данного объекта. Причем, это намер имеено элементов (т.е. у списка, например, этот номер равен кол-ву элементов списка).